In [ ]:
# CROP & FERTILIZER RECOMMENDATION

In [ ]:
#PREPROCESSING

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [5]:
# Step 1: Load dataset
df = pd.read_csv("Crop_recommendation.csv")

In [7]:
# Step 2: Basic info
print(df.head())
print(df['label'].value_counts())  # Crop distribution

    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice
label
rice           100
maize          100
jute           100
cotton         100
coconut        100
papaya         100
orange         100
apple          100
muskmelon      100
watermelon     100
grapes         100
mango          100
banana         100
pomegranate    100
lentil         100
blackgram      100
mungbean       100
mothbeans      100
pigeonpeas     100
kidneybeans    100
chickpea       100
coffee         100
Name: count, dtype: int64


In [9]:
# Step 3: Encode crop labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

In [11]:
# Save label mapping for decoding later
crop_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Crop Mapping:", crop_mapping)

Crop Mapping: {'apple': 0, 'banana': 1, 'blackgram': 2, 'chickpea': 3, 'coconut': 4, 'coffee': 5, 'cotton': 6, 'grapes': 7, 'jute': 8, 'kidneybeans': 9, 'lentil': 10, 'maize': 11, 'mango': 12, 'mothbeans': 13, 'mungbean': 14, 'muskmelon': 15, 'orange': 16, 'papaya': 17, 'pigeonpeas': 18, 'pomegranate': 19, 'rice': 20, 'watermelon': 21}


In [13]:
# Step 4: Define X and y
X = df[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
y = df['label_encoded']

In [15]:
# Step 5: Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [17]:
# Step 6: Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [ ]:
#CROP RECOMMENDATION MODEL

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [21]:
# Step 1: Initialize model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [23]:
# Step 2: Train the model
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [25]:
# Step 3: Make predictions
y_pred = rf_model.predict(X_test)

In [27]:
# Step 4: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.99


In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        26
           4       1.00      1.00      1.00        27
           5       1.00      1.00      1.00        17
           6       1.00      1.00      1.00        17
           7       1.00      1.00      1.00        14
           8       0.92      1.00      0.96        23
           9       1.00      1.00      1.00        20
          10       0.92      1.00      0.96        11
          11       1.00      1.00      1.00        21
          12       1.00      1.00      1.00        19
          13       1.00      0.96      0.98        24
          14       1.00      1.00      1.00        19
          15       1.00      1.00      1.00        17
          16       1.00      1.00      1.00        14
          17       1.00    

In [31]:
def predict_crop(N, P, K, temp, humidity, ph, rainfall):
    input_data = [[N, P, K, temp, humidity, ph, rainfall]]
    input_scaled = scaler.transform(input_data)
    pred_encoded = rf_model.predict(input_scaled)[0]
    pred_label = le.inverse_transform([pred_encoded])[0]
    return pred_label


result = predict_crop(90, 42, 43, 22, 80, 6.5, 200)
print("Suggested Crop:", result)


Suggested Crop: rice


In [ ]:
#FERTILIZER RECOMMENDATION

In [33]:
ideal_npk = {
    'rice':     {'N': 90, 'P': 40, 'K': 40},
    'maize':    {'N': 120, 'P': 60, 'K': 40},
    'sugarcane':{'N': 200, 'P': 50, 'K': 100},
    'cotton':   {'N': 75, 'P': 30, 'K': 40},
    'wheat':    {'N': 100, 'P': 50, 'K': 50},
    'banana':   {'N': 200, 'P': 60, 'K': 250},
    # Add more crops as needed
}

In [35]:
#FERTILIZER SUGGESTION FUNCTION 
def recommend_fertilizer(crop, N, P, K):
    crop = crop.lower()
    
    if crop not in ideal_npk:
        return "No fertilizer recommendation available for this crop."

    ideal = ideal_npk[crop]
    recommendations = []

    if N < ideal['N']:
        recommendations.append("Add Nitrogen-rich fertilizer (e.g., Urea)")
    elif N > ideal['N']:
        recommendations.append("Reduce Nitrogen input")

    if P < ideal['P']:
        recommendations.append("Add Phosphorus-rich fertilizer (e.g., SSP, DAP)")
    elif P > ideal['P']:
        recommendations.append("Reduce Phosphorus input")

    if K < ideal['K']:
        recommendations.append("Add Potassium-rich fertilizer (e.g., MOP)")
    elif K > ideal['K']:
        recommendations.append("Reduce Potassium input")

    return recommendations or ["NPK levels are balanced for this crop."]

In [37]:
crop = predict_crop(90, 42, 43, 22, 80, 6.5, 200)
fertilizer_advice = recommend_fertilizer(crop, 90, 42, 43)

#print("Crop Suggestion:", crop)
print("Fertilizer Advice:", fertilizer_advice)

Fertilizer Advice: ['Reduce Phosphorus input', 'Reduce Potassium input']


In [39]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.99


In [45]:
crop = predict_crop(85, 32, 38, 22, 80, 7.8, 190)
fertilizer_advice = recommend_fertilizer(crop, 92, 48, 43)

print("Crop Suggestion:", crop)
print("Fertilizer Advice:", fertilizer_advice)

Crop Suggestion: rice
Fertilizer Advice: ['Reduce Nitrogen input', 'Reduce Phosphorus input', 'Reduce Potassium input']


In [1]:
import pickle
import os

# Create folder if it doesn't exist
os.makedirs("models", exist_ok=True)

# Save the model
with open("models/crop_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Save the scaler
with open("models/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

# Save the label encoder
with open("models/label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)


NameError: name 'model' is not defined